In [239]:
method_name = 'SinglePeak_Scaled'
catchment_name = 'LinDyke'

In [240]:
# import os
# from pathlib import Path
# from PIL import Image
# # import geopandas as gpd
# import itertools
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
# import seaborn as sns
# from branca.element import Template, MacroElement
# from matplotlib import cm
# import earthpy.spatial as es

from my_functions import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define whether to filter out values <0.1
remove_little_values = True

In [241]:
if catchment_name == 'LinDyke':
    catchment_name_str = "Resampled.Terrain" 
    catchment_shpfilename = 'CatchmentLinDyke_exported'
    centre_location = [53.768306874761016, -1.3756056884868098]
    
elif catchment_name == 'WykeBeck':
    catchment_name_str = "Terrain.wykeDEM" 
    catchment_shpfilename = 'WykeBeckCatchment'
    centre_location = [53.81001289603593, -1.483594591196338]
    
if method_name == 'Idealised':
    most_extreme_str,least_extreme_str = '6h_sp_bl_0.9',"6h_sp_fl_0.1"
elif method_name == 'Observed':
    most_extreme_str,least_extreme_str = '6h_c5',"6h_c8"        
elif method_name == 'SinglePeak_Scaled':
    most_extreme_str,least_extreme_str = '6h_sp_+0%', "6h_sp_+20%"         

In [242]:
# Set up link to model directory and read in catchment shapefile
model_directory = '../../../FloodModelling/{}Models/Model_{}Profiles/'.format(catchment_name, method_name)
catchment_shp = model_directory + "{}.shp".format(catchment_shpfilename)
catchment_gdf = gpd.read_file(catchment_shp)
# boundaries_gpd_dict={}
# boundaries_gpd_dict['LinDyke'] =catchment_gdf

### Read in data for FL01 and BL09

In [243]:
# Read in data
most_extreme = prep_for_folium_plotting(model_directory+ '{}/Depth (Max).{}.tif'.format(most_extreme_str,catchment_name_str))
print(model_directory+ '{}/Depth (Max).{}.tif'.format(most_extreme_str,catchment_name_str))
least_extreme = prep_for_folium_plotting(model_directory+ '{}/Depth (Max).{}.tif'.format(least_extreme_str,catchment_name_str))
print(model_directory+ '{}/Depth (Max).{}.tif'.format(least_extreme_str,catchment_name_str))

# ree values
most_extreme.data[(most_extreme.data < 0.1)] = np.nan
least_extreme.data[(least_extreme.data < 0.1)] = np.nan

../../../FloodModelling/LinDykeModels/Model_SinglePeak_ScaledProfiles/6h_sp_+0%/Depth (Max).Resampled.Terrain.tif
../../../FloodModelling/LinDykeModels/Model_SinglePeak_ScaledProfiles/6h_sp_+20%/Depth (Max).Resampled.Terrain.tif


### Create versions of both scenarios where any flooded cell (>0.1m) has a value of 1, and the rest have 0

In [244]:
# most_extreme_binary = most_extreme.copy()
# most_extreme_binary.values[(most_extreme_binary.values >=0.1)] = 1
# most_extreme_binary.values[(most_extreme_binary.values <0.1)] = 0
# most_extreme_binary.values[np.isnan(most_extreme_binary.values)] = 0

# least_extreme_binary = least_extreme.copy()
# least_extreme_binary.values[(least_extreme_binary.values >=0.1)] = 1
# least_extreme_binary.values[(least_extreme_binary.values <0.1)] = 0
# least_extreme_binary.values[np.isnan(least_extreme_binary.values)] = 0

### Find the difference: 
i.e. cells have a value of:  
-1 if FL01 is bigger than BL09 (darkred)  
0 if there is no difference (either flooded in both scenarios or neither)  
1 if BL09 is bigger than FL01 (darkolivegreen)  

In [245]:
# diff_binary = most_extreme_binary -least_extreme_binary
# np.unique(diff_binary)

### Change the 0 values to np.nan (so they don't have a colour on the map)

In [246]:
# least_extreme_binary.values[(least_extreme_binary.values ==0)] = np.nan
# most_extreme_binary.values[(most_extreme_binary.values ==0)] = np.nan
# diff_binary.values[(diff_binary.values ==0)] = np.nan

### Find the difference in values

In [247]:
diff= most_extreme -least_extreme

### Convert this into a measure of whether the difference is positive or negative
0 = diff <0 (LEAST EXTREME bigger than MOST EXTREME) Red  
1 = diff == 0 (LEAST EXTREME==MOST EXTREME) Grey  
2 = diff >0 (MOST EXTREME bigger than LEAST EXTREME) Green  

In [248]:
pos_neg_depth_diff = np.where((np.isnan(diff)), np.nan, 
                          np.where((diff < 0), 0,
                            np.where((diff == 0), 1,
                                np.where((diff > 0), 2, 500))))

np.unique(pos_neg_depth_diff,return_counts=True)

(array([ 0.,  1.,  2., nan]),
 array([  1250196,         1,      4202, 112735476]))

### Create colourmaps

In [270]:
np.nanmax(most_extreme)

19.811218

In [271]:
# # Create discrete cmap
# colors_list = [ "red", "grey", "green"]
# cmap_posneg = mpl.colors.ListedColormap(colors_list)
# import branca.colormap as cm
# cmap_min=round(np.nanmin(diff.data),1)
# cmap_max=round(np.nanmax(diff.data),1)
cmap = mpl.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"])
colormap = cm.LinearColormap(colors=["red","violet","blue"], index=[np.nanmin(most_extreme),np.nanmax(most_extreme)],
                             vmin=np.nanmin(most_extreme),vmax=np.nanmax(most_extreme))

# from branca.colormap import linear
# x=linear.YlOrRd_09.scale(cmap_min,cmap_max)
# x

# # colors_list_tinary = [ "darkred", "darkolivegreen"]
# # cmap_tinary = mpl.colors.ListedColormap(colors_list_tinary)

# colors_list_binary =  [ "red","green"]
# cmap_binary = mpl.colors.ListedColormap(colors_list_binary)

### Plot

In [250]:
# data  = ma.masked_invalid(diff)
# np.histogram(data.data)
# plt.hist(data.data[np.isfinite(data.data)])

#cmap = cm.LinearColormap(cm.magma_r.colors,vmin=0.3, vmax=1)

#cmap_func = lambda x: ImageColor.getcolor(cmap(x), 'RGBA') if ~np.isnan(x) else (0,0,0,0)

# folium.raster_layers.ImageOverlay(numpy_array,
# opacity=0.6,
# bounds=map_bounds
# colormap=cmap_func).add_to(map)

In [272]:
colormap

In [274]:
# Set up figure
f = folium.Figure(width=1000, height=1000)

# Create base map - location figures were from clat, clon, but wanted to create map before loop
mapa = folium.Map(location=centre_location,zoom_start=13).add_to(f)
folium.TileLayer(tiles = 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
    attr="No Base Map",name="No Base Map",show=True).add_to(mapa)

# Catchment boundary
catchment_boundary_feature_group = FeatureGroup(name=catchment_name)
catchment_boundary_feature_group.add_child(folium.GeoJson(data=catchment_gdf["geometry"], style_function=lambda x, 
                                     fillColor='#00000000', color='Black': { "fillColor": '#00000000',"color": 'Black',}))
mapa.add_child(catchment_boundary_feature_group)

###########################################
# Get coordinates needed in plotting
clat, clon = most_extreme.y.values.mean(), most_extreme.x.values.mean()
mlat, mlon = most_extreme.y.values.min(), most_extreme.x.values.min()
xlat, xlon = most_extreme.y.values.max(), most_extreme.x.values.max()
###########################################

# Pos Neg depth difference
###########################################
data  = ma.masked_invalid(pos_neg_depth_diff)
colored_data = colorize(data.data, cmap = cmap_posneg)
feature_group1 = FeatureGroup(name='PosNegDepthDiff')
feature_group1.add_child(folium.raster_layers.ImageOverlay(colored_data,[[mlat, mlon], [xlat, xlon]],
                          opacity=0.8,interactive=True, popup='Test'))
mapa.add_child(feature_group1)

# Actual values
###########################################
data  = ma.masked_invalid(least_extreme)
colored_data = colorize(data.data, cmap = colormap)
feature_group2 = FeatureGroup(name='MostExtreme')
feature_group2.add_child(folium.raster_layers.ImageOverlay(colored_data,[[mlat, mlon], [xlat, xlon]],
                          opacity=0.8,interactive=True, popup='Test'))
mapa.add_child(feature_group2)

data  = ma.masked_invalid(most_extreme)
colored_data = colorize(data.data, cmap = colormap)
feature_group3 = FeatureGroup(name='LeastExtreme')
feature_group3.add_child(folium.raster_layers.ImageOverlay(colored_data,[[mlat, mlon], [xlat, xlon]],
                          opacity=0.8,interactive=True, popup='Test'))
mapa.add_child(feature_group3)

###########################################
# Mask out nans
# data  = ma.masked_invalid(diff)
# colored_data = colorize(data.data, cmap = cmap)
# feature_group5 = FeatureGroup(name='Difference (actual)')
# feature_group5.add_child(folium.raster_layers.ImageOverlay(colored_data,[[mlat, mlon], [xlat, xlon]],
#                           opacity=0.8,interactive=True, popup='Difference (actual)'))
# mapa.add_child(feature_group5)

# Add layers to map
mapa.add_child(LayerControl("topright", collapsed = False))
display(mapa)

ValueError: <branca.colormap.LinearColormap object at 0x7f186454e400> is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'ci

### Read in landcover data

In [253]:
# landcover_binary_fp =  model_directory +"LandCover_clipped_reclassified.tif"
# landcover_full_fp =  model_directory +"LandCover_clipped.tif"

### Read in flood data

In [254]:
# fp_feh= '../../../../FloodModelling/MeganModel_New/6h_feh_sp/{} (Max).Resampled.Terrain.tif'
# fp_sp_10 = '../../../../FloodModelling/MeganModel_New/6h_sp_+10/{} (Max).Resampled.Terrain.tif'
# fp_b2 = '../../../../FloodModelling/MeganModel_New/6h_sp_bl_0.9/{} (Max).Resampled.Terrain.tif'
# fp_f2= '../../../../FloodModelling/MeganModel_New/6h_sp_fl_0.1/{} (Max).Resampled.Terrain.tif'
# fp_f1= '../../../../FloodModelling/MeganModel_New/6h_sp_fl_0.3/{} (Max).Resampled.Terrain.tif'

# depth_b2, out_meta = prepare_rainfall_scenario_raster(fp_b2.format('Depth'), remove_little_values)
# depth_feh = prepare_rainfall_scenario_raster(fp_feh.format('Depth'), remove_little_values)[0]
# depth_sp_10 = prepare_rainfall_scenario_raster(fp_sp_10.format('Depth'), remove_little_values)[0]
# depth_f2 = prepare_rainfall_scenario_raster(fp_f2.format('Depth'), remove_little_values)[0]
# depth_f1 = prepare_rainfall_scenario_raster(fp_f1.format('Depth'), remove_little_values)[0]

### Set all values over 0.1 to 1, and rest to 0

In [255]:
# depth_feh = np.where(depth_feh>=0.1, 1, 0)
# depth_sp_10= np.where(depth_sp_10>=0.1, 1, 0)
# depth_b2 = np.where(depth_b2 >=0.1, 1, 0)
# depth_f2 = np.where(depth_f2>=0.1, 1, 0)
# depth_f1 = np.where(depth_f1 >=0.1, 1, 0)

### Find difference between the FEH and another profile (ones which are off the 1:1 line on scatter plot).

In [256]:
# diff_feh_b2 = depth_b2 - depth_feh 
# diff_f1_b2 = depth_b2 - depth_f1 

### Set 0s to np.nan ()

In [257]:
# depth_feh = np.where(depth_feh==0, np.nan, depth_feh)
# depth_sp_10 = np.where(depth_sp_10==0, np.nan, depth_sp_10)
# depth_b2 = np.where(depth_b2==0, np.nan, depth_b2)
# depth_f1 = np.where(depth_f1==0, np.nan, depth_f1)

### Test whether processing has worked as expected

In [258]:
# def colorize(array, cmap):
#     cm = plt.cm.get_cmap(cmap)    
#     if np.nanmin(array) == np.nanmax(array):
#         return np.uint8(cm(array)  * 255)
#     else:
#         normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array)) 
#         return np.uint8(cm(normed_data)  * 255)


In [259]:
# np.nanmax(diff[np.isfinite(diff)])
# np.histogram(depth_f1[np.isfinite(depth_f1)])
# np.unique(depth_f1)

### Where difference is 0, set to np.nan

In [260]:
# # Difference in scatter plot
# diff_feh_b2 = np.where(diff_feh_b2==0, np.nan, diff_feh_b2)
# # No difference in scatter plot
# diff_f1_b2 = np.where(diff_f1_b2==0, np.nan, diff_f1_b2)

### Define colourmaps

In [261]:
# cmap_landcover_binary = mpl.colors.ListedColormap(['grey', 'white'])
# cmap_diff = mpl.colors.ListedColormap(['purple', 'darkorange', 'green'])
# cmap = mpl.colors.ListedColormap(['white', 'blue'])
# cmap_landcover_full = mpl.colors.ListedColormap(['darkslategrey', 'khaki', 'darkgreen','darkturquoise','#59F74D', 'slateblue', 'lightgreen', 'palegreen', 'pink'])

### Save rasters (needed for plotting)

In [262]:
# # save_array_as_raster(diff_feh_b2, '../../../../FloodModelling/MeganModel_New/diff_feh_b2.tif', out_meta) 
# # save_array_as_raster(diff_f1_b2, '../../../../FloodModelling/MeganModel_New/diff_f1_b2.tif', out_meta) 

# save_array_as_raster(depth_sp_10, '../../../../FloodModelling/MeganModel_New/depth_sp_10_test.tif', out_meta) 
# save_array_as_raster(depth_feh, '../../../../FloodModelling/MeganModel_New/depth_feh_test.tif', out_meta) 
# save_array_as_raster(depth_b2, '../../../../FloodModelling/MeganModel_New/depth_b2_test.tif', out_meta) 
# save_array_as_raster(depth_f1, '../../../../FloodModelling/MeganModel_New/depth_f1_test.tif', out_meta) 

### Set up for plotting

In [263]:
# # dict_of_fps_and_names = {'landcover_full': landcover_full_fp,
# #                         'landcover_binary': landcover_binary_fp,
# #                         "diff_f1_b2": '../../../../FloodModelling/MeganModel_New/diff_f1_b2.tif',
# #                          "diff_feh_b2": '../../../../FloodModelling/MeganModel_New/diff_feh_b2.tif',
# #                          "b2": '../../../../FloodModelling/MeganModel_New/depth_b2_test.tif',
# #                          "f1":'../../../../FloodModelling/MeganModel_New/depth_f1_test.tif',
# #                         "feh":'../../../../FloodModelling/MeganModel_New/depth_feh_test.tif'}    


# dict_of_fps_and_names = {'landcover_binary': landcover_binary_fp,
#                          "sp10":"../../../../FloodModelling/MeganModel_New/depth_sp_10_test.tif",
#                         "feh":"../../../../FloodModelling/MeganModel_New/depth_feh_test.tif"}    


# dict_of_cmaps ={'landcover_binary': cmap_landcover_binary, "sp10":cmap, 'landcover_full': cmap_landcover_full, "diff_feh_b2":cmap_diff,
#                 "diff_f1_b2":cmap_diff,"b2":cmap, "feh":cmap , "f1":cmap }

### Plot with folium

In [264]:
# # Set up figure
# f = folium.Figure(width=800, height=700)

# # Create base map - location figures were from clat, clon, but wanted to create map before loop
# mapa = folium.Map(location=[53.768306874761016, -1.3756056884868098],zoom_start=13).add_to(f)
# folium.TileLayer(
#     opacity=1,
#     tiles = 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
#     attr="No Base Map",
#     name="No Base Map",
#     show=True
# ).add_to(mapa)

# # Add to map
# catchment_boundary_feature_group = FeatureGroup(name='Catchment boundary')
# catchment_boundary_feature_group.add_child(folium.GeoJson(data=catchment_gdf["geometry"], style_function=lambda x, 
#                                                           fillColor='#00000000', color='Black': {
#         "fillColor": '#00000000',"color": 'Black',}))

# # Add raster data
# for name,fp in dict_of_fps_and_names.items():
    
#     # read in with xarray and convert projection
#     xarray_dataarray = prep_for_folium_plotting(fp)
#     # Get coordinates needed in plotting
#     clat, clon = xarray_dataarray.y.values.mean(), xarray_dataarray.x.values.mean()
#     mlat, mlon = xarray_dataarray.y.values.min(), xarray_dataarray.x.values.min()
#     xlat, xlon = xarray_dataarray.y.values.max(), xarray_dataarray.x.values.max()
#     # Apply colormap
#     data  = ma.masked_invalid(xarray_dataarray.values)
#     colored_data = colorize(data.data, cmap=dict_of_cmaps[name])
#     # Add to map
#     feature_group1 = FeatureGroup(name=name)
#     feature_group1.add_child(folium.raster_layers.ImageOverlay(colored_data,
#                               [[mlat, mlon], [xlat, xlon]],
#                               opacity=1,interactive=True, popup=name))
#     mapa.add_child(feature_group1)

# # Add legend
# macro = MacroElement()
# macro._template = Template(template)
# mapa.get_root().add_child(macro)

# # Add layer control button
# mapa.add_child(catchment_boundary_feature_group)
# mapa.add_child(LayerControl("topright", collapsed = False))
# display(mapa)    

In [265]:
template = """
{% macro html(this, kwargs) %}
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            left: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});
  </script>
</head>
<body>
 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 20px; bottom: 20px;'>
<div class='legend-title'> Key </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#800080.;opacity:1;'></span> B2 flooded (not FEH)</li>
    <li><span style='background:#008000;opacity:1;'></span> FEH flooded (not B2)</li>
  </ul>
</div>
</div>
 
</body>
</html>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""